### Easy

Оценить с помощью доверительных интервалов:

выживаемость всех пассажиров (оценить среднее по isSurvived)

выживаемость женщин

выживаемость людей с каютами первого класса


Ответить на вопросы:

какой вариант оценки доверительных интервалов нужно использовать в данной задаче и почему?

какая из ошибок более критична для данной задачи - I или II рода?

In [56]:
import numpy as np
import pandas as pd
import scipy

In [57]:
df = pd.read_csv('train.csv')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [58]:
alpha = 0.05
n = df.Survived.count()
mean = df.Survived.mean()
std = df.Survived.std()

In [59]:
from scipy.stats import t

In [60]:
t.interval(1 - alpha, n - 1, loc=mean, scale=std)

(-0.5711640365064437, 1.3388408041832114)

It still doesn't work :(

In [61]:
#Пробуем применить предложенные поправки
upd = std / np.sqrt(n)
upd

0.016301460571865715

In [62]:
t.interval(1 - alpha, n - 1, loc=mean, scale=upd)

(0.35184459896802966, 0.41583216870873796)

Now it works!

In [63]:
#Функция для расчета этого параметра на будущее:
def upd(std, n):
    return std / np.sqrt(n)


In [64]:
#Выживаемость женщин
f_df = df.loc[df['Sex'] == 'female']
f_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [65]:
alpha = 0.05
n = f_df.Survived.count()
mean = f_df.Survived.mean()
std = f_df.Survived.std()
t.interval(1 - alpha, n - 1, loc=mean, scale=upd(std, n))

(0.6933807726141715, 0.7906956605068475)

In [66]:
#Выживаемость людей с каютами первого класса
df1 = df.loc[df['Pclass'] == 1]
df1.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S


In [67]:
alpha = 0.05
n = df1.Survived.count()
mean = df1.Survived.mean()
std = df1.Survived.std()
t.interval(1 - alpha, n - 1, loc=mean, scale=upd(std, n))

(0.5647152190969751, 0.6945440401622842)

Ответы на вопросы:

1. Используем вариант оценки, когда дисперсия не известна, при необходимости дорабатывая аргумент scale.

2. Исходя из допущения, что лучше "спасти" погибшего, чем не спасти выжившего, критичнее ошибка II рода.

### Normal
Проверить гипотезу о том, что женщины выживают чаще мужчин с уровнем значимости 0.05

Проверить гипотезу о том, что дети выживают чаще, чем взрослые

In [68]:
from scipy.stats import ttest_ind

Вывод: т.к. pvalue меньше уровня значимости, H0 отвергаем, H1 верна: женщины выживают чаще

H0: Женщины и мужчины выживают с равной вероятностью

H1: Женщины выживают чаще

In [69]:
f = df[df['Sex'] == 'female'].Survived.to_numpy()
m = df[df['Sex'] == 'male'].Survived.to_numpy()

In [70]:
scipy.stats.ttest_ind(f, m)

Ttest_indResult(statistic=19.29781655012335, pvalue=1.4060661308802594e-69)

H0: Дети и взрослые выживают с равной вероятностью

H1: Дети выживают чаще

In [71]:
c = df[df['Age'] < 18].Survived.to_numpy()
a = df[df['Age'] >= 18].Survived.to_numpy()

In [72]:
scipy.stats.ttest_ind(c, a)

Ttest_indResult(statistic=3.171077486504452, pvalue=0.0015838873450969102)

Вывод: т.к. pvalue меньше уровня значимости, H0 отвергаем, H1 верна: дети выживают чаще